# Train an IntegratedML model on Marketing Campaign Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [ ]:
!pip install jupyterthemes

In [ ]:
!jt -r

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

!ls /tf/intersystems-jdbc-3.1.0.jar
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

In [ ]:
!pip install JayDeBeApi

In [ ]:
!pip list JPype1

### 1. Set environment variables, if necessary

In [ ]:
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

### 2. Get jdbc connection and cursor

In [1]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:51773/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [2]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

### 3. specify the source data table

In [3]:
dataTable = 'SQLUser.Campaign'

### 4. Execute a query and display results in Pandas DataFrame

In [20]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

,AGE,AMOUNT,COUNTRY_OF_BIRTH,EDUCATION,EMPLOYMENT,GENDER,INV_EARNINGS,INV_LOSSES,MARITAL_STATUS,OCCUPATION,PROSPECT_ID,RACE,RELATIONSHIP,RESPONSE,WORKWEEK_LENGTH,YEARS_EDUC
0,50,123.90,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001107,WHITE,HUSBAND,0,40,9
1,49,97.91,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,LOGISTICSTRANSP,77001140,WHITE,HUSBAND,0,40,9
2,37,135.65,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001143,WHITE,HUSBAND,0,40,9
3,43,93.23,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,CLEANINGHANDLING,77001187,WHITE,HUSBAND,0,40,9
4,54,143.72,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,LOGISTICSTRANSP,77001412,WHITE,HUSBAND,0,40,9
5,27,232.20,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,SERVICEVARIOUS,77001429,WHITE,HUSBAND,0,40,9
6,29,103.46,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001443,WHITE,HUSBAND,0,40,9
7,35,265.20,USA,COLLEGE,BUSINESS,MALE,0,0,DIVORCED,ADMINOFFICE,77047141,WHITE,UNMARRIED,0,40,10
8,30,291.64,USA,COLLEGE,BUSINESS,MALE,0,0,NOMARRIAGE,ADMINOFFICE,77047153,WHITE,NOFAMILY,0,40,10
9,29,255.78,USA,COLLEGE,BUSINESS,MALE,0,0,MARRIEDCIVSPO,ADMINOFFICE,77047160,WHITE,HUSBAND,0,40,10


In [21]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

,Aggregate_1
0,48842


### 5. Make some views to split training and testing datasets

In [ ]:
# Small training set view
curs.execute("CREATE VIEW SQLUser.CampaignTrainingSmall AS SELECT * FROM SQLUser.Campaign WHERE ID<3001")
# Small prediction set
curs.execute("CREATE VIEW SQLUser.CampaignPredictSmall AS SELECT * FROM SQLUser.Campaign WHERE ID>47000")

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [14]:
curs.execute("CREATE MODEL CampaignModel PREDICTING (RESPONSE) FROM CampaignTrainingSmall")

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [15]:
curs.execute("TRAIN MODEL CampaignModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [23]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,CampaignModel,CampaignModel2,AutoML,2020-06-19 01:06:40.633000,classification,"ModelType:Logistic Regression, Package:sklearn..."


### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [24]:
df4 = pd.read_sql("SELECT PREDICT(CampaignModel) AS PredictedResponse, \
                  response AS ActualResponse FROM CampaignPredictSmall", conn)
display(df4)

,PredictedResponse,Actualresponse
0,1,0
1,1,0
2,1,1
3,1,0
4,1,1
...,...,...
1837,1,0
1838,0,0
1839,0,0
1840,0,0


### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [27]:
curs.execute("VALIDATE MODEL CampaignModel FROM SQLUser.CampaignPredictSmall")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
CampaignModel21,0.75,0.73,0.68,0.8
CampaignModel22,0.75,0.73,0.68,0.8
